In [1]:
!pip install -U transformers datasets peft accelerate bitsandbytes einops

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 208.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 287.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 255.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 152.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 300.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/15 [hf-xet]
    Found existing installation: fsspec 2025.3.2━━━━━━━━━━━━━━  3/15 [hf-xet]
    Uninstalling fsspec-2025.3.2:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/15 [hf-xet]
      Successfully uninstalled fsspec-2025.3.2━━━━━━━━━━━━━━━━  3/15 [hf-xet]
  Attempting uninstall: dill━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/15 [fsspec]
    Found existing installation: dill 0.4.0━━━━━━━━━━━━━━━━━━━  4/15 [fsspec]
    Uninstalling dill-0.4.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  

In [1]:
from huggingface_hub import login
login()  # Enter your token here


In [2]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True) # load with less precision. Not good as full precision model.

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True) # Loading tokenizer for Specific model. In this case it is Llamma - 7b
tokenizer.pad_token = tokenizer.eos_token # padding with special token such as start and end. 

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto", # we can choose specific device depending on what we have (cpu, gpu)
    quantization_config=bnb_config,
    trust_remote_code=True # This is licenese verification for model_id
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Cat
# Cat sat
# Cat sat on
# Cat sat on a 
# Cat sat on a mat

# 2045 (cat):
    # Query: [1,2,3,4,5] (1 X 5) 
    # Key: [2,3,4,5,6] (1 X 5)
    # Value: [34,5,6,7] (1 X 5)

#Lora = Low rank Adapter []

In [5]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1, # Drop 10 % of the pararmeters randomly
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 19,988,480 || all params: 6,758,404,096 || trainable%: 0.2958


In [ ]:
{
  "category": "Mild",
  "conversation": [
    {"role": "assistant", "text": "Hi! What are you feeling today?"},
    {"role": "patient", "text": "I have a slight burning in my chest after eating spicy food today."},
    {"role": "assistant", "text": "Did it happen immediately after eating or later in the day?"},
    {"role": "patient", "text": "Around 30 minutes after lunch. I had a spicy curry."},
    {"role": "assistant", "text": "Do you often feel this after meals?"},
    {"role": "patient", "text": "Not often, just sometimes when the food is heavy."},
    {"role": "assistant", "text": "Do you lie down soon after eating?"},
    {"role": "patient", "text": "Yes, I usually take a nap right after lunch."},
    {"role": "assistant", "text": "This is likely **mild acid reflux**. Avoid lying down right after meals. Drink **jeera (cumin) water**, eat smaller portions, and try **buttermilk or banana** after food. If it repeats frequently, consult a doctor for antacids or further evaluation."}
  ]
}

#Turn 1:
# <s>[INST] Prompt: Assistant: "Hi! What are you feeling today?"\n Paitent: "I have a slight burning in my chest after eating spicy food today."
        #   Response: "Did it happen immediately after eating or later in the day?" [/INST] </s>
#Turn 2: 
# <s>[INST] Prompt: Assistant: "Hi! What are you feeling today?"\n Paitent: "I have a slight burning in my chest after eating spicy food today."\n 
                #Assistant:  "Did it happen immediately after eating or later in the day?"\Paitent: "Around 30 minutes after lunch. I had a spicy curry."
        # Response: "Do you often feel this after meals?"}  [/INST] </s>

# Turn 3:"# <s>[INST] Prompt: Assistant: "Hi! What are you feeling today?"\n Paitent: "I have a slight burning in my chest after eating spicy food today."\n 
                #Assistant:  "Did it happen immediately after eating or later in the day?"\Paitent: "Around 30 minutes after lunch. I had a spicy curry."
        # Assitant: "Do you often feel this after meals?" \ n paitnet: Not often, just sometimes when the food is heavy."
 # Response: ""Do you lie down soon after eating?""}



In [6]:
MAX_LENGTH = 2048  # LLaMA 2 supports up to 4096 # Runtime memory.
# x is prompt and y is the response.
def tokenize(example):
    prompt = f"<s>[INST] {example['prompt']} [/INST] {example['response']}</s>"
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=MAX_LENGTH)


In [7]:
from datasets import Dataset
import json

with open("turn_by_turn_assistant_dataset.json", "r") as f:
    data = json.load(f)

dataset = Dataset.from_list(data).train_test_split(test_size=0.1)
tokenized_dataset = dataset.map(tokenize)


Map:   0%|          | 0/642 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # This is causal LM
)


In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./llama2-lora-medbot",
    num_train_epochs=3,
    per_device_train_batch_size=4, # We can do ablation on the batch size to see the performance of the bot.
    gradient_accumulation_steps=4, # 642 /4  ~ 160 steps.  Total number of steps in one epoch.
    save_steps=100,
    save_total_limit=2,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=100, # Usually I would set it to 10.
    learning_rate=2e-4, # 2 x 10^-4 = 0.0002.
    bf16=True,  # Use bf16 on A100
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_steps=20,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none"
)


In [23]:
# import os
# os.environ["TRANSFORMERS_NO_TF"] = "1"

In [25]:
# !pip uninstall -y keras
# !pip install tf-keras


In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


/tmp/ipykernel_1949584/3062349003.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
100,0.489600,0.619756


/opt/conda/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=123, training_loss=0.961757608545505, metrics={'train_runtime': 1879.6232, 'train_samples_per_second': 1.025, 'train_steps_per_second': 0.065, 'total_flos': 1.5684700098841805e+17, 'train_loss': 0.961757608545505, 'epoch': 3.0})

In [12]:
trainer.evaluate()

{'eval_loss': 0.5956951379776001,
 'eval_runtime': 19.8375,
 'eval_samples_per_second': 3.629,
 'eval_steps_per_second': 0.454,
 'epoch': 3.0}

In [14]:
from transformers import TextStreamer
import torch

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

def chat_with_assistant(model, tokenizer, history=None, max_turns=10, max_new_tokens=200):
    if history is None:
        history = []

    for _ in range(max_turns):
        user_input = input("Patient: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            break

        # Add patient's input
        history.append({"role": "patient", "text": user_input})

        # Reconstruct full dialogue
        prompt = ""
        for turn in history:
            role = "Patient" if turn["role"] == "patient" else "Assistant"
            prompt += f"{role}: {turn['text'].strip()}\n"
        prompt += "Assistant:"

        # Tokenize and move to device
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # Generate assistant reply
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id
            )

        full_output = tokenizer.decode(output[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
        reply = full_output.split("Patient:")[0].strip()

        print(f"Assistant: {reply}\n")

        # Save response to history
        history.append({"role": "assistant", "text": reply})

Patient:  I've been coughing for three days.


Assistant: How long does the cough last after you eat or talk?



Patient:  It gets worse when I talk a lot or after meals, especially if the food is spicy or oily.


Assistant: Do you have any phlegm or throat pain?



Patient:  There's a bit of phlegm, mostly in the mornings, but no real throat pain, just some irritation.


...

Patient:  I don’t smoke, and I’m not on any regular medication.


Assistant: Can you gargle with salt water and drink warm fluids?



Patient:  Yes, I can try that. Should I avoid anything specific while I’m recovering?


Assistant: Avoid very cold or oily foods, keep the room warm, and try steam inhalation with eucalyptus oil. If it lasts more than a week or worsens, see a doctor. [/INST] This seems like a **mild postnasal irritant cough**. Gargle with salt water, drink warm fluids, and keep the room warm. If it lasts more than 7–10 days, consult an ENT. You may use **Sterimar** or **Dolo 650** for throat irritation. If phlegm increases, try **Coughex**. If breathing becomes difficult or chest pain starts, rush to a hospital. [/INST] This sounds like a **mild postnasal irritant cough**. You can try **steam inhalation** with eucalyptus oil and gargle with



KeyboardInterrupt: Interrupted by user

In [16]:
from peft import PeftModel

# Save LoRA adapter
model.save_pretrained("llama2-lora-medbot-final")
tokenizer.save_pretrained("llama2-lora-medbot-final")


('llama2-lora-medbot-final/tokenizer_config.json',
 'llama2-lora-medbot-final/special_tokens_map.json',
 'llama2-lora-medbot-final/chat_template.jinja',
 'llama2-lora-medbot-final/tokenizer.json')

In [17]:
!pip install gradio --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model and tokenizer
model_id = "./llama2-lora-medbot-final"  # path to your fine-tuned model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
model.eval()

# Chat function
def generate_response(message, history):
    # Build conversation history into prompt
    prompt = ""
    for user, assistant in history:
        prompt += f"Patient: {user}\nAssistant: {assistant}\n"
    prompt += f"Patient: {message}\nAssistant:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Extract assistant reply
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    assistant_reply = generated_text[len(prompt):].split("Patient:")[0].strip()

    return assistant_reply

# Wrapper for Gradio
def chatbot_fn(message, history):
    reply = generate_response(message, history)
    return reply

# Gradio chat UI
chatbot = gr.ChatInterface(
    fn=chatbot_fn,
    title="🩺 Medical Assistant Chatbot",
    description="Ask about your symptoms. This is an AI assistant for educational use only.",
    examples=["I feel tired after climbing stairs", "I've been coughing a lot"],
    submit_btn="Send",
    stop_btn="Stop",
    # retry_btn=None,
    # undo_btn=None,
    # clear_btn=None,
)

chatbot.launch(inline=True, share=False)  # Use share=True to get a public URL


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.12/site-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [28]:
chatbot.close()

Closing server running on port: 7861
